# El Niño / Southern Oscillation (ENSO) from Sea Surface Temperature (SST)

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#El-Niño-/-Southern-Oscillation-(ENSO)-from-Sea-Surface-Temperature-(SST)" data-toc-modified-id="El-Niño-/-Southern-Oscillation-(ENSO)-from-Sea-Surface-Temperature-(SST)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>El Niño / Southern Oscillation (ENSO) from Sea Surface Temperature (SST)</a></span><ul class="toc-item"><li><span><a href="#Learning-Objectives" data-toc-modified-id="Learning-Objectives-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Learning Objectives</a></span></li><li><span><a href="#Access-data-directly-from-NOAA-using-OpenDAP-protocol" data-toc-modified-id="Access-data-directly-from-NOAA-using-OpenDAP-protocol-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Access data directly from NOAA using OpenDAP protocol</a></span></li><li><span><a href="#Select-only-the-more-recent-part-of-the-data." data-toc-modified-id="Select-only-the-more-recent-part-of-the-data.-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Select only the more recent part of the data.</a></span></li><li><span><a href="#Interactive-Plot" data-toc-modified-id="Interactive-Plot-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Interactive Plot</a></span></li><li><span><a href="#Calculate-Climatology-and-Monthly-Anomaly" data-toc-modified-id="Calculate-Climatology-and-Monthly-Anomaly-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Calculate Climatology and Monthly Anomaly</a></span></li><li><span><a href="#Plot-global-mean" data-toc-modified-id="Plot-global-mean-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Plot global mean</a></span></li><li><span><a href="#Calculate-Oceanic-Niño-Index" data-toc-modified-id="Calculate-Oceanic-Niño-Index-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Calculate Oceanic Niño Index</a></span></li><li><span><a href="#Composite-the-global-SST-on-the-Niño3.4-index" data-toc-modified-id="Composite-the-global-SST-on-the-Niño3.4-index-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>Composite the global SST on the Niño3.4 index</a></span></li><li><span><a href="#Calculate-Empirical-Orthogonal-Functions" data-toc-modified-id="Calculate-Empirical-Orthogonal-Functions-1.9"><span class="toc-item-num">1.9&nbsp;&nbsp;</span>Calculate Empirical Orthogonal Functions</a></span></li><li><span><a href="#Going-Further" data-toc-modified-id="Going-Further-1.10"><span class="toc-item-num">1.10&nbsp;&nbsp;</span>Going Further</a></span></li></ul></li></ul></div>




**Author:** [Ryan Abernathey](http://rabernat.github.io)

According to [NOAA](https://www.esrl.noaa.gov/psd/enso/):
>  El Niño and La Niña, together called the El Niño Southern Oscillation (ENSO), are periodic departures from expected sea surface temperatures (SSTs) in the equatorial Pacific Ocean. These warmer or cooler than normal ocean temperatures can affect weather patterns around the world by influencing high and low pressure systems, winds, and precipitation. ENSO may bring much needed moisture to a region while causing extremes of too much or too little water in others.


## Learning Objectives

In this notebook, we use the python [xarray](http://xarray.pydata.org/en/latest/) package to examine SST data from NOAA's [Extended Reconstructed Sea Surface Temperature (ERSST) v5](https://www.ncdc.noaa.gov/data-access/marineocean-data/extended-reconstructed-sea-surface-temperature-ersst-v5) product.
We use [holoviews](http://holoviews.org/) to interactively visualize the data.
We then demonstrate how easy it is to calculate the [Niño3.4 index](https://www.ncdc.noaa.gov/teleconnections/enso/indicators/sst/).
Finally, we use the [EOFS](https://ajdawson.github.io/eofs/index.html) package to compute the Emprical Orthogoal Functions of global SST.

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import xarray as xr
from matplotlib import pyplot as plt
import numpy as np
import hvplot.xarray
import holoviews as hv
hv.extension('bokeh')
%matplotlib inline

## Access data directly from NOAA using OpenDAP protocol

OPeNDAP is a means by which a user can access a remote file over the internet. Xarray has has OPeNDAP capabilities:

In [ ]:
url = 'http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/noaa.ersst.v5/sst.mnmean.nc'
ds = xr.open_dataset(url)
ds

## Select only the more recent part of the data.

In [ ]:
ds = ds.sel(time=slice('1960', '2019'))
ds

In [ ]:
# size of data in MB
ds.nbytes/1e6 

In [ ]:
# actually download the data
ds.load()

## Interactive Plot

In [ ]:
%output holomap='scrubber' fps=3
ds.sst.hvplot('lon', 'lat', cmap='Magma').redim.range(sst=(-2, 30))

## Calculate Climatology and Monthly Anomaly

Xarray makes this particularly easy

In [ ]:
sst_clim = ds.sst.groupby('time.month').mean(dim='time')
sst_anom = ds.sst.groupby('time.month') - sst_clim

Detrend signal

In [ ]:
from scipy.signal import detrend
sst_anom_detrended = xr.apply_ufunc(detrend, sst_anom.fillna(0),
                                    kwargs={'axis': 0}).where(~sst_anom.isnull())

## Plot global mean

In [ ]:
# For a global average, we need to weigh the points by cosine of latitude.
# This is not built into xarray because xarray is not specific to geoscientific data.
weights = np.cos(np.deg2rad(ds.lat)).where(~sst_anom[0].isnull())
weights /= weights.mean()

In [ ]:
(sst_anom * weights).mean(dim=['lon', 'lat']).plot(label='raw')
(sst_anom_detrended * weights).mean(dim=['lon', 'lat']).plot(label='detrended')
plt.grid()
plt.legend()

In [ ]:
sst_anom_detrended.hvplot('lon', 'lat', cmap='RdBu_r').redim.range(sst=(-2, 2))

## Calculate Oceanic Niño Index

In [ ]:
sst_anom_nino34 = sst_anom_detrended.sel(lat=slice(5, -5), lon=slice(190, 240))
sst_anom_nino34_mean = sst_anom_nino34.mean(dim=('lon', 'lat'))
oni = sst_anom_nino34_mean.rolling(time=3, center=True).mean()

Plot it:

In [ ]:
oni.plot()
plt.grid()
plt.ylabel('Anomaly (dec. C)');

Compare to the [official version](https://www.ncdc.noaa.gov/teleconnections/enso/indicators/sst/) from NOAA:

![NOAA Nino 3.4](../../../assets/noaa-nino-sst.gif)

## Composite the global SST on the Niño3.4 index

In [ ]:
positive_oni = ((oni>0.5).astype('b').rolling(time=5, center=True).sum()==5)
negative_oni = ((oni<0.5).astype('b').rolling(time=5, center=True).sum()==5)
positive_oni.astype('i').plot(marker='.', label='positive')
(-negative_oni.astype('i')).plot(marker='.', label='negative')
plt.legend()

In [ ]:
sst_anom.where(positive_oni).mean(dim='time').plot()
plt.title('SST Anomaly - Positive Niño3.4');

In [ ]:
sst_anom.where(negative_oni).mean(dim='time').plot()
plt.title('SST Anomaly - Negative Niño3.4');

## Calculate Empirical Orthogonal Functions

Using (EOFs) empirical orthogonal functions is a common technique to decompose a signal varying in time and space into a form that is easier to interpret in terms of spatial and temporal variance. 

In [ ]:
from eofs.xarray import Eof
solver = Eof(sst_anom_detrended, weights=np.sqrt(weights))
eof1 = solver.eofsAsCorrelation(neofs=1)
pc1 = solver.pcs(npcs=1, pcscaling=1)

In [ ]:
eof1.sel(mode=0).plot()

The first principal component is almost identical to the ONI.

In [ ]:
pc1.sel(mode=0).plot(label='PC mode 0')
(-oni).plot(label='- ONI')
plt.grid()
plt.legend()

## Going Further

- [eofs package documentation](https://ajdawson.github.io/eofs/latest/index.html)
- [hvPlot documentation](https://hvplot.pyviz.org/)
- [holoviews documentation](http://holoviews.org/)